In [ ]:
%load_ext autoreload
%autoreload 2
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
from pandas import Timedelta
# from optiml.utils import sf
import time
from optiml.utils.sf import logger, sql_to_df, run_sql, conn, session
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

import plotly
plotly.offline.init_notebook_mode()

In [ ]:
run_sql("set lookback_days=300")

sql = """
with qhe as (
select 
    *    
from
    qhe_lookback
)
,
stats as (
select
    hour(start_time) as hour_start,
    warehouse_name,
    wh_utilization_status,
    count(*) num_queries,
    sum(query_cost) total_query_cost,
    max_by(start_time, warehouse_size) as latest_size
from qhe
group by 
    hour_start,
    warehouse_name,
    wh_utilization_status
)
select 
    *,
    100*ratio_to_report(num_queries) over (partition by warehouse_name, hour_start) as pct_of_total_queries,
    100*ratio_to_report(total_query_cost) over (partition by warehouse_name, hour_start) as pct_total_query_cost,
    sum(num_queries) over (partition by warehouse_name, hour_start) as total_warehouse_queries,
    sum(total_query_cost) over (partition by warehouse_name, hour_start) as total_warehouse_query_cost
    
from stats
order by (wh_utilization_status != 'right-sized') desc, total_query_cost desc;
"""

import itables


utilization_by_hour = sql_to_df(sql)
itables.show(utilization_by_hour)

In [ ]:
df = utilization_by_hour[utilization_by_hour.warehouse_name == 'DATA_CLEANSING_WH']
import plotly.express as px

fig = px.bar(df, x='hour_start', y='num_queries', color='wh_utilization_status')
fig.show()

fig = px.bar(df, x='hour_start', y='pct_of_total_queries', color='wh_utilization_status')
fig.show()

fig = px.bar(df, x='hour_start', y='pct_total_query_cost', color='wh_utilization_status')
fig.show()


In [ ]:
df